# Detekce, popis a klasifikace obličeje z RGB obrazu
Druhá část cvičení se zaměří na detekci obličeje pomocí tradičních metod zpracování obrazu a následně na popis a klasifikaci detekovaného obličeje pomocí hlubokých neuronových sítí. Soudobé přístupy oba tyto úkoly typicky spojují do jedné sítě, ale o těch se zmíníme až na konci...

## Stažení modelů
Do složky s tímto notebookem si stáhněte VGG model, které budeme později používat:

 https://www.dropbox.com/s/183mgti3tdfmu9d/VGG_FACE_pyTorch_small.t7?dl=1

## Detekce obličeje
Prvním úkolem je ve vstupním obrazu detekovat umístění obličeje nebo obličejů. Jednak nám to umožní vyfiltrovat nedůležité části obrazu před dalším zpracováním a typicky tak zpracování urychlit a zpřesnit, ale především nám to pomůže při výskytu více obličejů v jednom obrázku. Metody popisu totiž chceme spouštět zaručeně jen na jednom obličeji...

Jenže, jak na to? Zkusme si představit jak vypadá typický obličej. Oči a obočí jsou tmavší než tváře, stejně tak jsou tmavší než nos. Jenže modelovat explicitně podobu obličeje je poměrně pracné. Především, když můžeme použít hrubou sílu a nechat si vymodelovat typický obličej ne na úrovni pixelů, ale na úrovni rozdílů intenzit vhodně zvolených oblastí napříč rozlišeními.

K tomu je možné použít například Haarovu funkci všemožně naškálovanou a narotovanou jak je podrobněji popsáno v článku Paula Violy a Michaela Jonese: "Rapid Object Detection using a Boosted Cascade of Simple Features" a klasifikátor, který bude detekovat ty oblasti obrázku, které obsahují předučenou kombinaci rozdílů intenzit podle vybraných funkcí.

<img src="images/haar_wavelet.svg" width=40% style="float:left"> <img src="images/haar_application.png" width=40%>
<div style="clear:both"/>

Pro učení klasifikátoru je zapotřebí algoritmu dodat velké množství obrázků obličeje (a obrázků, které obličej neobsahují), na kterých proběhne, zjednodušeně řečeno, určení hodnot všech Haarových funkcí a výběr funkcí (a prahů) vhodných pro rozlišení zda obrázek obsahuje nebo neobsahuje obličej. Tím vznikne model, který je při dostatečných schopnostech generalizace možné následně používat pro detekci i neznámých obličejů. Model zde z dat učit nebudeme, použijeme model již natrénovaný pro detekci obličeje dívajícího se přímo do kamery, který je distribuovaný s OpenCV. Pro jistotu jej ale přikládáme přímo do složky s notebookem.

### Importy knihoven a modelu

In [ ]:
import cv2
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

### Příprava obrázku a detekce
1) Načtěte pomocí knihovny OpenCV (a případně dalších knihoven) obrázek `who_is_this.jpg`, převeďte do RGB pro další zpracování a do odstínů šedi pro detekci Haarovou kaskádou.

2) Detekujte obličeje. Uvedené parametry funkce detectMultiScale specifikují úroveň škálování a citlivost filtru.

Pro další práci zachovejte jen první detekovaný obličej, vizualizujte jeho detekci.

In [ ]:
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

face = 

3) Oblast obličeje ořízněte, zkontrolujte zda je výřez čtvercový, případně upravte.

In [ ]:
cropped = 

## Popis a klasifikace obličeje
Nyní když máme obličej nahrubo označený, zajímalo by nás kdo se za ním skrývá.

Opět je pro tyto úlohy používáno velké množství hrubé síly v podobě hlubokých neuronových sítí. Neuronové sítě (dále NN - neural networks) jsou klasifikátory inspirované přírodou, respektive nervovou tkání. Jen pro připomenutí, ta je tvořena jednotlivými (vzájemně velmi podobnými) neurony, které na jedné straně přijímají signál od ostatních neuronů (pomocí různě širokých vstupních kanálů v dendritech, v NN šířkám kanálů říkáme váhy), signál mohou velmi jednoduchými operacemi upravit a při překročení kritické hladiny (prahu) signál transformovaný aktivační funkcí předat dalším neuronům.

![](images/neuron.svg)

Pro zjednodušení používáme typicky několik omezujících podmínek. Nejčastěji používané sítě jsou dopředné (feed-forward). Neuron tedy může posílat signál pouze směrem vpřed, do další vrstvy; což je druhé zjednodušení. Neurony organizujeme do takzvaných vrstev, kdy všechny neurony v jedné vrstvě mají stejný typ a aktivační funkci (pozor, ne váhy a práh, ty jsou učené nezávisle).

Jedinou drobnou komplikací je, že pro zpracování dat s kontextem je vhodné používat konvoluční neuronové sítě (CNN). Konvoluci (1D) snad nemusím připomínat, konvoluční vrstva je pak jen aplikací tohoto principu ("jedeme" okénkem po obrázku a vracíme okénkem váženou sumu intenzit pixelů).

Pro shrnutí, stačí znát několik základních pojmů:
- **vstupní vrstva** - vrstva neuronové sítě, která nemá žádné vstupy a výstupy těchto neuronů jsou nastaveny na hodnoty samotných dat
- **výstupní vrstva** - vrstva neuronové sítě, která nemá výstupy ale obsahuje po "protečení" dat sítí požadovanou výstupní hodnotu
- **plně propojená (fully connected) vrstva** - vrstva, ve které jsou všechny neurony napojené na všechny neurony předchozí vrstvy
- **konvoluční (convolutional) vrstva** - vrstva, která počítá konvoluci nad vstupními daty, trénují se tak pro každý výstupní filtr "jen" váhy konvolučních jader a jeden práh
- **sdružovací (pooling) vrstva** - vrstva redukující dimenzionalitu bez trénovatelných parametrů
- **softmax vrstva** - vrstva, které vektor reálných čísel transformuje na distribuci pravděpodobnosti

Postup učení umělých neuronových sítí není dobře možné shrnout do několikaminutového tutoriálu, tak si představme, že síť považujeme za velkou černou krabici ze které trčí mnoho set tisíc až milionů potenciometrů. Všechny parametry sítě (včetně "tvaru" sítě) jsou laděny tak, aby pro data na vstupu (trénovací množinu, v našem případě intenzity jednotlivých pixelů v jednotlivých kanálech) síť vracela požadovaný výstup (pravděpodobnosti tříd, tedy v našem případě jmen) a pro negativní příklady vracela nuly. Pro představu pomůže velmi dobře stravitelné video od CGP Grey: https://www.youtube.com/watch?v=wvWpdrfoEv0

Neuronová síť je jen tak dobrá, jak dobře je vyladěný její model. K tomu jsou potřeba obrovská množství dat. A model pak stejně rozpozná jen to, na co byl natrénován. Takže celou síť od začátku zde trénovat také nebudeme. Ale pokud by Vás zajímalo jaké modely jsou jednoduše dostupné pro PyTorch: http://www.robots.ox.ac.uk/~albanie/pytorch-models.html

A když už jsem to nakousl, Torch je jednou z knihoven, která umožňuje uživatelsky přívětivou práci s neuronovými sítěmi (resp. knihovnou THNN), včetně akcelerace na GPU a jejích tenzorových jádrech, což je pro rozumně rychlé (rozuměj: pro větší problémy vůbec možné) trénování neuronových sítí klíčové. Jenže bychom se museli naučit jazyk Lua. Naštěstí má implementované vazby i pro Python v podobě balíčku PyTorch. Jen aby nepřekvapilo, že budeme používat funkci `load_lua`.

### Načtení knihoven a modelu
V našem případě použijeme model uložený ve formátu t7 (pro luatorch), který obsahuje jak informace o tvaru sítě, tak nastavení vah, prahů, aktivačních funkcí, atd. Jen 32. modul trochu opravíme...

In [ ]:
import torch
from torch.legacy import nn
from torch.utils.serialization import load_lua

vgg_face = load_lua("VGG_FACE_pyTorch_small.t7")
vgg_face.modules[31] = nn.View(1, 25088)

Síť VGG16 vypadá takto:
![](images/vgg16.png)

Námi používaná síť VGG-Face má v poslední, klasifikační vrstvě, 2622 neuronů, jejichž koncové hodnoty odpovídají pravděpodobnosti klasifikace do jmen v souboru `names.txt`.

### Příprava obličeje a klasifikace + popis
1) Nejprve je potřeba oříznutý obličej upravit do podoby, kterou vyžaduje síť.

Zmenšete oříznutý obrázek na velikost 224x224 pixelů.

Pak přesuňte osy tak, aby první osou byl barevný kanál, následně osy y a x (v pořadí běžném pro OpenCV). Ideálně k přesunu osy využijte vhodnou funkci NumPy.

In [ ]:
cropped = 

2) Od obrázku je nutné odečíst ještě průměrnou hodnotu kanálů použitou při trénování sítě.

Tento krok byl použit proto, aby absolutní hodnota vah v síti byla minimální.

Vstup pro síť ještě vhodně rozbalíme.

In [ ]:
cropped = cropped.astype(float)
training_mean = [129.1863, 104.7624, 93.5940]

for i in range(3):
    cropped[i] = cropped[i] - training_mean[i]

nn_input = torch.Tensor(crop).unsqueeze(0)

3) Výstup zajistíme dopředným spuštěním sítě

Najděte pomocí vhodné funkce knihovny NumPy index maximální hodnoty ve výstupním vektoru. Ověřte, které celebritě patří a s jakou pravděpodobností síť třídu přiřadila.

In [ ]:
nn_output = vgg_face.forward(nn_input)

prediction = 

Speciální význam v síti VGG-Face (i VGG16 obecně) má předposlední vrstva (s indexem 35). Ta obsahuje vektor o délce 4096, který reprezentuje embedding zkoumaného obličeje do prostoru tzv. Eigenfaces - prototypů obličejů. I proto jsou tyto sítě široce používané pro předzpracování obrazových dat do prostoru o výrazně nižší dimenzi. Pro porovnání dvou obličejů je možné použít L2 vzdálenost nad normovanými vektory.

## Úkol
Nyní máte všechny potřebné ingredience.

Z popisu obsaženého v předposlední vrstvě VGG-Face a s využitím binárního lineárního klasifikátoru (např. LinearSVC) z knihovny Scikit-Learn naučte klasifikátor, který od sebe rozpozná Vás a Vašeho souseda.

## OpenFace
Tento přístup má jistě mnohá úskalí. Pokud se dotyčný nedívá přímo do kamery nebo pokud má výrazně nakloněnou hlavu, detekce ani popis nemusí fungovat správně. Poté je potřeba dělat trochu více magie, kterou nechcete psát stále dokola. A proto vznikla knihovna OpenFace: https://cmusatyalab.github.io/openface/

Doporučuji vyzkoušet.